# Performance

This notebook measures performance of `DiscreteEvents.jl` functionality in order to compile the [performance section](https://pbayer.github.io/DiscreteEvents.jl/dev/performance/) of the documentation.

In [1]:
using DiscreteEvents, BenchmarkTools, Random
res = Dict(); # results dictionary

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1278


## Event-based simulations

The following is a modification of the [channel example](https://pbayer.github.io/DiscreteEvents.jl/dev/approach/#Event-based-modeling-1). We simulate events 

1. taking something from a common channel or waiting if there is nothing, 
2. then taking a delay, doing a calculation and
3. returning three times to the first step.

As calculation we take the following Machin-like sum:

$$4 \sum_{k=1}^{n} \frac{(-1)^{k+1}}{2 k - 1}$$

This gives a slow approximation to $\pi$. The benchmark creates long queues of timed and conditional events and measures how fast they are handled.

### Function calls as events

The first implementation is based on events with `fun`s. 

In [4]:
function take(id::Int64, qpi::Vector{Float64}, step::Int64)
    if isready(ch)
        take!(ch)                                         # take something from common channel
        event!(fun(put, id, qpi, step), after, rand())    # timed event after some time
    else
        event!(fun(take, id, qpi, step), fun(isready, ch)) # conditional event until channel is ready
    end
end

function put(id::Int64, qpi::Vector{Float64}, step::Int64)
    put!(ch, 1)
    qpi[1] += (-1)^(id+1)/(2id -1)      # Machin-like series (slow approximation to pi)
    step > 3 || take(id, qpi, step+1)
end

function setup(n::Int)                     # a setup the simulation
    resetClock!(𝐶)
    Random.seed!(123)
    global ch = Channel{Int64}(32)  # create a channel
    global qpi = [0.0]
    si = shuffle(1:n)
    for i in 1:n
        take(si[i], qpi, 1)
    end
    for i in 1:min(n, 32)
        put!(ch, 1) # put first tokens into channel 1
    end
end

setup (generic function with 1 method)

If we setup 250 summation elements, we get 1000 timed events and over 1438 sample steps with conditional events.

In [5]:
@time setup(250)
println(@time run!(𝐶, 500))
println("result=", qpi[1])

  0.016450 seconds (21.52 k allocations: 1.044 MiB)
  0.256077 seconds (472.22 k allocations: 25.350 MiB, 5.88% gc time)
run! finished with 1000 clock events, 1468 sample steps, simulation time: 500.0
result=3.137592669589458


In [6]:
t = run(@benchmarkable run!(𝐶, 500) setup=setup(250) evals=1 seconds=15.0 samples=50)

BenchmarkTools.Trial: 
  memory estimate:  10.45 MiB
  allocs estimate:  196603
  --------------
  minimum time:     26.848 ms (0.00% GC)
  median time:      28.240 ms (0.00% GC)
  mean time:        28.804 ms (2.21% GC)
  maximum time:     40.416 ms (0.00% GC)
  --------------
  samples:          50
  evals/sample:     1

In [7]:
res["Event based with fun"] = minimum(t).time * 1e-6 # ms 

26.847644

### Expressions as events

The 2nd implementation does the same but with expressions, which are `eval`uated in global scope during runtime. This gives a one-time warning for beeing slow:

In [8]:
function take(id::Int64, qpi::Vector{Float64}, step::Int64)
    if isready(ch)
        take!(ch)                                            # take something from common channel
        event!(:(put($id, qpi, $step)), after, rand())   # timed event after some time
    else
        event!(:(take($id, qpi, $step)), :(isready(ch))) # conditional event until channel is ready
    end
end

function put(id::Int64, qpi::Vector{Float64}, step::Int64)
    put!(ch, 1)
    qpi[1] += (-1)^(id+1)/(2id -1)      # Machin-like series (slow approximation to pi)
    step > 3 || take(id, qpi, step+1)
end

put (generic function with 1 method)

In [9]:
@time setup(250)
println(@time run!(𝐶, 500))
println("result=", sum(qpi))

  0.000083 seconds (1.54 k allocations: 92.047 KiB)


┌ Warning: Evaluating expressions is slow, use functions instead
└ @ DiscreteEvents /Users/paul/.julia/packages/DiscreteEvents/SpY5t/src/fclosure.jl:28


  9.065884 seconds (6.04 M allocations: 382.416 MiB, 0.53% gc time)
run! finished with 1000 clock events, 1468 sample steps, simulation time: 500.0
result=3.137592669589458


In [10]:
t = run(@benchmarkable run!(𝐶, 500) setup=setup(250) evals=1 seconds=15.0 samples=50)

BenchmarkTools.Trial: 
  memory estimate:  378.62 MiB
  allocs estimate:  5964325
  --------------
  minimum time:     8.948 s (0.46% GC)
  median time:      8.968 s (0.43% GC)
  mean time:        8.968 s (0.43% GC)
  maximum time:     8.988 s (0.39% GC)
  --------------
  samples:          2
  evals/sample:     1

In [11]:
res["Event based with Expressions"] = minimum(t).time * 1e-6 #
res

Dict{Any,Any} with 2 entries:
  "Event based with fun"         => 26.8476
  "Event based with Expressions" => 8947.51

In [13]:
res["Event based with Expressions"]/res["Event based with fun"]

333.2699188055384

This takes much longer and shows that `eval` for Julia expressions, done in global scope is very expensive and should be avoided if performance is any issue.

### Involving a global variable

The third implementation works with `fun`s like the first but involves a global variable `A`:

In [14]:
function take(id::Int64, qpi::Vector{Float64}, step::Int64)
    if isready(ch)
        take!(ch)                                       # take something from common channel
        event!(fun(put, id, qpi, step), after, rand())    # timed event after some time
    else
        event!(fun(take, id, qpi, step), fun(isready, ch)) # conditional event until channel is ready
    end
end

function put(id::Int64, qpi::Vector{Float64}, step::Int64)
    put!(ch, 1)
    global A += (-1)^(id+1)/(2id -1)      # Machin-like series (slow approximation to pi)
    step > 3 || take(id, qpi, step+1)
end

function setup(n::Int)                     # a setup he simulation
    resetClock!(𝐶)
    Random.seed!(123)
    global ch = Channel{Int64}(32)  # create a channel
    global A = 0
    si = shuffle(1:n)
    for i in 1:n
        take(si[i], qpi, 1)
    end
    for i in 1:min(n, 32)
        put!(ch, 1) # put first tokens into channel 1
    end
end

setup (generic function with 1 method)

In [15]:
ch = Channel{Int64}(32)
@code_warntype put(1, qpi, 1)

Variables
  #self#::Core.Compiler.Const(put, false)
  id::Int64
  qpi::Array{Float64,1}
  step::Int64

Body::Union{Nothing, Bool, DiscreteEvents.Register{DiscreteEvents.DiscreteEvent{DiscreteEvents.var"#7#8"{typeof(put)},Nothing}}}
1 ─       Main.put!(Main.ch, 1)
│         nothing
│   %3  = (id + 1)::Int64
│   %4  = ((-1) ^ %3)::Int64
│   %5  = (2 * id)::Int64
│   %6  = (%5 - 1)::Int64
│   %7  = (%4 / %6)::Float64
│   %8  = (Main.A + %7)::Any
│         (Main.A = %8)
│   %10 = (step > 3)::Bool
└──       goto #3 if not %10
2 ─       return %10
3 ─ %13 = (step + 1)::Int64
│   %14 = Main.take(id, qpi, %13)::Union{Nothing, DiscreteEvents.Register{DiscreteEvents.DiscreteEvent{DiscreteEvents.var"#7#8"{typeof(put)},Nothing}}}
└──       return %14


In [16]:
@time setup(250)
println(@time run!(𝐶, 500))
println("result=", A)

  0.000171 seconds (2.29 k allocations: 64.609 KiB)
  0.040990 seconds (207.64 k allocations: 10.937 MiB)
run! finished with 1000 clock events, 1468 sample steps, simulation time: 500.0
result=3.137592669589458


In [17]:
t = run(@benchmarkable run!(𝐶, 500) setup=setup(250) evals=1 seconds=10.0 samples=30)

BenchmarkTools.Trial: 
  memory estimate:  10.48 MiB
  allocs estimate:  198603
  --------------
  minimum time:     27.621 ms (0.00% GC)
  median time:      29.558 ms (0.00% GC)
  mean time:        29.853 ms (1.90% GC)
  maximum time:     39.512 ms (0.00% GC)
  --------------
  samples:          30
  evals/sample:     1

In [18]:
res["Event based with functions and a global variable"] = minimum(t).time * 1e-6 #
res

Dict{Any,Any} with 3 entries:
  "Event based with fun"                             => 26.8476
  "Event based with Expressions"                     => 8947.51
  "Event based with functions and a global variable" => 27.6208

In this case the compiler does well to infer the type of `A` and it runs only marginally slower than the first version.